### <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#294B8E; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #294B8E">Проект для «Викишоп»</p>

<a id="1.2"></a>
<div style="padding:20px;
            color:white;
            margin:10;
            font-size:170%;
            text-align:left;
            display:fill;
            border-radius:5px;
            background-color:#294B8E;
            overflow:hidden;
            font-weight:700"><span style='color:#CDA63A'>|</span> Содержание</div>

<div class="toc"><ul class="toc-item"><li><span><a href="#Описание-данных" data-toc-modified-id="Описание-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Описание данных</a></span></li>
    <li><span><a href="#Загрузка-библиотек" data-toc-modified-id="Загрузка-библиотек-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Загрузка библиотек</a></span></li>
    <li><span><a href="#EDA" data-toc-modified-id="EDA-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>EDA</a></span></li>
    <li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Подготовка данных</a></span></li>
    <li><span><a href="#Подготовим-выборки-для-обучения-моделей" data-toc-modified-id="Подготовим-выборки-для-обучения-моделей-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Подготовим выборки для обучения моделей</a></span></li>
    <li><span><a href="#Обучение-моделей" data-toc-modified-id="Обучение-моделей-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Обучение моделей</a></span></li>
       <li><span><a href="#Тестирование-лучшей-модели" data-toc-modified-id="Тестирование-лучшей-модели-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Тестирование лучшей модели</a></span></li>
    <li><span><a href="#Вывод" data-toc-modified-id="Вывод-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Вывод</a></span></li>

<a id="Описание-данных"></a>
<div style="padding:20px;
            color:white;
            margin:10;
            font-size:170%;
            text-align:left;
            display:fill;
            border-radius:5px;
            background-color:#294B8E;
            overflow:hidden;
            font-weight:700"><span style='color:#CDA63A'>|</span> Описание данных</div>

<ul>
<li>text - текст комментария</li>
<li>toxic - целевой признак</li>
</ul>


<div class="alert alert-block alert-info" style="background-color:#FFFAF0;
                                                 color:black;
                                                 border-color:#DEB887;
                                                 border-radius:5px;
                                                 width:50%;
                                                 margin: auto;
                                                 text-align: left;">


Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

</div>

<a id="Загрузка-библиотек"></a>
<div style="padding:20px;
            color:white;
            margin:10;
            font-size:170%;
            text-align:left;
            display:fill;
            border-radius:5px;
            background-color:#294B8E;
            overflow:hidden;
            font-weight:700"><span style='color:#CDA63A'>|</span> Загрузка библиотек</div>

In [1]:
!pip list | grep pymystem

pymystem3                         0.2.0


In [2]:
import os
import pandas as pd
import re
import numpy as np
from pymystem3 import Mystem 
import nltk
nltk.download('stopwords')
from nltk.corpus import wordnet
from catboost import CatBoostClassifier
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

/opt/conda/lib/python3.9/site-packages/joblib/_multiprocessing_helpers.py:45: UserWarning: [Errno 28] No space left on device.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<a id="EDA"></a>
<div style="padding:20px;
            color:white;
            margin:10;
            font-size:170%;
            text-align:left;
            display:fill;
            border-radius:5px;
            background-color:#294B8E;
            overflow:hidden;
            font-weight:700"><span style='color:#CDA63A'>|</span> EDA</div>

In [3]:
pth1 = '/datasets/toxic_comments.csv'
pth2 = 'toxic_comments.csv'
if os.path.exists(pth1):
    data = pd.read_csv(pth1)
elif os.path.exists(pth2):
    data = pd.read_csv(pth2)
else:
    print('Something is wrong')

In [4]:
data.head(5) 

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [5]:
data.tail(5) 

,Unnamed: 0,text,toxic
159287,159446,""":::::And for the second time of asking, when ...",0
159288,159447,You should be ashamed of yourself \n\nThat is ...,0
159289,159448,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,159449,And it looks like it was actually you who put ...,0
159291,159450,"""\nAnd ... I really don't think you understand...",0


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


<div class="alert alert-block alert-info" style="background-color:#FFFAF0;
                                                 color:black;
                                                 border-color:#DEB887;
                                                 border-radius:5px;
                                                 width:50%;
                                                 margin: auto;
                                                 text-align: left;">

<b>✍ Вывод</b>
        <ul>
<li>В данных пропусков нет.</li>
 
</ul>
</div>

<a id="Подготовка-данных"></a>
<div style="padding:20px;
            color:white;
            margin:10;
            font-size:170%;
            text-align:left;
            display:fill;
            border-radius:5px;
            background-color:#294B8E;
            overflow:hidden;
            font-weight:700"><span style='color:#CDA63A'>|</span> Подготовка данных</div>

In [7]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [11]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [8]:
#лемматизация
lemmatizer = WordNetLemmatizer()
def lemmatize(text):
    word_list = nltk.word_tokenize(text)
    lemm_text = ' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in word_list])
    return lemm_text

In [9]:
# напишем функцию, которая оставит в тексте только кириллические символы и пробелы
def clear_text(text):
    text = re.sub(r'[^a-zA-Z ]', ' ', text)
    return " ".join(text.split())

In [12]:
from tqdm.notebook import tqdm
tqdm.pandas()

data['lemm_text'] = data['text'].progress_apply(clear_text)
data['lemm_text'] = data['lemm_text'].progress_apply(lemmatize)

  0%|          | 0/159292 [00:00<?, ?it/s]

  0%|          | 0/159292 [00:00<?, ?it/s]

In [13]:
#вызовем функцию nltk_stopwords.words, передадим ей аргумент 'english', то есть англоязычные стоп-слова
stopwords = set(nltk_stopwords.words('english'))


<div class="alert alert-block alert-info" style="background-color:#FFFAF0;
                                                 color:black;
                                                 border-color:#DEB887;
                                                 border-radius:5px;
                                                 width:50%;
                                                 margin: auto;
                                                 text-align: left;">

<b>✍ Вывод</b>
        <ul>
<li>Текст леммитизирован и очищен.</li>
 
</ul>
</div>

<a id="Подготовим-выборки-для-обучения-моделей"></a>
<div style="padding:20px;
            color:white;
            margin:10;
            font-size:170%;
            text-align:left;
            display:fill;
            border-radius:5px;
            background-color:#294B8E;
            overflow:hidden;
            font-weight:700"><span style='color:#CDA63A'>|</span> Подготовим выборки для обучения моделей</div>

In [14]:
X_train, X_test, y_train, y_test = train_test_split(data['lemm_text'], data['toxic'], 
                                                    test_size=0.1,
                                                    random_state=42)
print(f"Размер тренировочного корпуса: {len(X_train)}")
print(f"Размер тренировочного корпуса: {len(X_test)}")

Размер тренировочного корпуса: 143362
Размер тренировочного корпуса: 15930


In [15]:
#создадим счётчик, указав в нём стоп-слова
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

In [16]:
#посчитаем TF-IDF для корпуса текстов функцией fit_transform() для обучающей выборки
tf_idf_X_train = count_tf_idf.fit_transform(X_train)


In [17]:
#посчитаем TF-IDF для корпуса текстов функцией fit_transform() для тестовой выборки
tf_idf_X_test = count_tf_idf.transform(X_test)

<div class="alert alert-block alert-info" style="background-color:#FFFAF0;
                                                 color:black;
                                                 border-color:#DEB887;
                                                 border-radius:5px;
                                                 width:50%;
                                                 margin: auto;
                                                 text-align: left;">

<b>✍ Вывод</b>
        <ul>
<li>Данные разделены на обучающую и тестовую выборки.</li>
 
</ul>
</div>

<a id="Обучение-моделей"></a>
<div style="padding:20px;
            color:white;
            margin:10;
            font-size:170%;
            text-align:left;
            display:fill;
            border-radius:5px;
            background-color:#294B8E;
            overflow:hidden;
            font-weight:700"><span style='color:#CDA63A'>|</span> Обучение моделей</div>

### <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#294B8E; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #294B8E">Logistic regression</p>

In [ ]:
prm_logr = {'C': np.logspace(-4, 4, 20),
             'max_iter': [1000]}
logr = LogisticRegression()
grid_search = GridSearchCV(logr, 
                           prm_logr,
                           cv=3,
                           scoring='f1',
                           n_jobs=-1,
                           verbose=2)
grid_search.fit(tf_idf_X_train, y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV] END ............................C=0.0001, max_iter=1000; total time=   3.3s
[CV] END ............................C=0.0001, max_iter=1000; total time=   3.1s
[CV] END ............................C=0.0001, max_iter=1000; total time=   3.2s
[CV] END ............C=0.00026366508987303583, max_iter=1000; total time=   4.1s
[CV] END ............C=0.00026366508987303583, max_iter=1000; total time=   4.2s
[CV] END ............C=0.00026366508987303583, max_iter=1000; total time=   3.9s
[CV] END .............C=0.0006951927961775605, max_iter=1000; total time=   4.5s
[CV] END .............C=0.0006951927961775605, max_iter=1000; total time=   4.4s
[CV] END .............C=0.0006951927961775605, max_iter=1000; total time=   4.3s
[CV] END .............C=0.0018329807108324356, max_iter=1000; total time=   5.8s
[CV] END .............C=0.0018329807108324356, max_iter=1000; total time=   5.5s
[CV] END .............C=0.0018329807108324356, m

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_score_

### <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#294B8E; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #294B8E">SGD</p>

In [ ]:
model_sgd = SGDClassifier(random_state=42)

params_sgd = {
    'max_iter':[500,1000],
    'loss':['log','squared_hinge'],
    'n_iter_no_change':[20,30]}

In [ ]:
grid_sgd = GridSearchCV(model_sgd, 
                        param_grid=params_sgd, 
                        scoring='f1',
                        verbose=3,
                        cv=5, 
                        n_jobs=-1)
grid_sgd.fit(tf_idf_X_train, y_train)

In [ ]:
grid_sgd.best_params_

In [ ]:
grid_sgd.best_score_

<div class="alert alert-block alert-info" style="background-color:#FFFAF0;
                                                 color:black;
                                                 border-color:#DEB887;
                                                 border-radius:5px;
                                                 width:50%;
                                                 margin: auto;
                                                 text-align: left;">

<b>✍ Вывод</b>
        <ul>
<li>Найдены лучшие гиперпараметры для соотвествующих моделей с помощью GridSearchCV.</li>
<li>При сравнении моделей с помощью атрибута best_score_ лучшей моделью стала логистическая регрессия.</li>
 
</ul>
</div>

<a id="Тестирование-лучшей-модели"></a>
<div style="padding:20px;
            color:white;
            margin:10;
            font-size:170%;
            text-align:left;
            display:fill;
            border-radius:5px;
            background-color:#294B8E;
            overflow:hidden;
            font-weight:700"><span style='color:#CDA63A'>|</span> Тестирование лучшей модели</div>

In [ ]:
logr = LogisticRegression(C=11.288378916846883, max_iter=1000, random_state=12345)
logr.fit(tf_idf_X_train, y_train)
predict_logr = logr.predict(tf_idf_X_test)
print('f1 логистической регрессии:', f1_score(y_test, predict_logr))

<a id="Вывод"></a>
<div style="padding:20px;
            color:white;
            margin:10;
            font-size:170%;
            text-align:left;
            display:fill;
            border-radius:5px;
            background-color:#294B8E;
            overflow:hidden;
            font-weight:700"><span style='color:#CDA63A'>|</span> Вывод</div>

<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">
<br>📑Было проведено исследование, для<b> интернет-магазин «Викишоп» </b>. 
<br>Задача: обучить модель классифицировать комментарии на позитивные и негативные. 
        

<br><b> Входные данные - данных с разметкой о токсичности правок. </b>

        
<br> 📊 Текст очищен и лемматизирован.
        
<br> 📊 Были обучены модели логистической регрессии и стохастического градиентного спуска. 
        
<br> 📊 Найдены лучшие гиперпараметры для соотвествующих моделей с помощью GridSearchCV.
        
<br> 📊 Показано сравнение качества исследованных моделей с помощью атрибута best_score_. Лучшая модель - логистическая регрессия.
 
<br>Исходя из анализа, сделан вывод, что исходя из данных лучше всего с предсказанием токсичных комментариев справилась модель логистической регрессии. Что нам показывает как f1 оценка.  